In [52]:
import numpy as np
import pandas as pd
import json
import re
import nltk
import pickle
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from collections import Counter

from psutil import users
from sentence_transformers import SentenceTransformer

In [59]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
Data preprocessing module for job recommendation system.

This module handles the preprocessing of user and job data for the recommendation system,
including text cleaning, skill parsing, and embedding generation for skills and titles.
"""

import numpy as np
import pandas as pd
import json
import re
import nltk
import pickle
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from collections import Counter
from sentence_transformers import SentenceTransformer


class DataPreprocessor:
    """
    A class for preprocessing job and user data.

    Handles the cleaning, parsing, and vectorization of job and user data
    for use in recommendation algorithms.

    Attributes
    ----------
    stop_words : set
        Set of English stop words from NLTK
    lemmatizer : WordNetLemmatizer
        NLTK lemmatizer for text preprocessing
    """

    def __init__(self):
        """
        Initialize the DataPreprocessor with stop words and lemmatizer.

        Loads stopwords from a local pickle file if available,
        otherwise downloads them from NLTK and saves for future use.
        """
        self.lemmatizer = WordNetLemmatizer()

        try:
            import os
            stopwords_file = 'nltk_stopwords.pkl'
            if os.path.exists(stopwords_file):
                with open(stopwords_file, 'rb') as f:
                    self.stop_words = pickle.load(f)
                print("Loaded stopwords from local file")
            else:
                try:
                    self.stop_words = set(stopwords.words('english'))
                except LookupError:
                    nltk.download('stopwords')
                    self.stop_words = set(stopwords.words('english'))
                with open(stopwords_file, 'wb') as f:
                    pickle.dump(self.stop_words, f)
                print("Downloaded stopwords and saved to local file")
        except Exception as e:
            print(f"Error handling stopwords file: {e}")
            try:
                self.stop_words = set(stopwords.words('english'))
            except LookupError:
                nltk.download('stopwords')
                self.stop_words = set(stopwords.words('english'))

    def get_user_skills(self, skill_dicts):
        """
        Extract skill names from a list of skill dictionaries.

        Parameters
        ----------
        skill_dicts : list
            List of dictionaries containing skill information

        Returns
        -------
        list
            List of skill names, or empty list if input is None or invalid
        """
        if not skill_dicts:
            return []
        skills = []
        for skill_dict in skill_dicts:
            if isinstance(skill_dict, dict) and 'name' in skill_dict and isinstance(skill_dict['name'], str):
                skills.append(skill_dict['name'])
        return skills

    def parse_skills(self, skills_list):
        """
        Parse a list of skill strings into a list of skill phrases.

        Parameters
        ----------
        skills_list : list
            List of strings containing concatenated skills

        Returns
        -------
        list
            List of parsed skill phrases, or empty list if input is None or invalid
        """
        def clean_string(skill_list):
            if not skill_list:
                return []
            clean_list = []
            for s in skill_list:
                if isinstance(s, str):
                    clean_list.append(re.sub(r'\s*\(.*?\)', '', s))
            return clean_list

        def standardize_skill(skill):
            if not isinstance(skill, str):
                return ""
            skill = skill.lower()
            skill = re.sub(r'[^\w\s]', '', skill)
            return skill.strip()

        def parse_skill_list(skills_list):
            skills = []
            for skills_string in skills_list:
                if not isinstance(skills_string, str):
                    continue
                words = skills_string.split()
                words = [word for word in words if word not in self.stop_words]

                current_skill = []
                prev_starts_with_capital = False

                for word in words:
                    if word:
                        starts_with_capital = word[0].isupper()
                        if starts_with_capital:
                            skill_phrase = ' '.join(current_skill)
                            if skill_phrase:
                                std_skill_phrase = standardize_skill(skill_phrase)
                                if std_skill_phrase:
                                    skills.append(std_skill_phrase)
                            current_skill = [word]
                        else:
                            current_skill.append(word)
                        prev_starts_with_capital = starts_with_capital

                if current_skill:
                    skill_phrase = ' '.join(current_skill)
                    if skill_phrase:
                        std_skill_phrase = standardize_skill(skill_phrase)
                        if std_skill_phrase:
                            skills.append(std_skill_phrase)

            return skills

        cleaned = clean_string(skills_list)
        parsed_skills = parse_skill_list(cleaned)
        return parsed_skills

    def preprocess_text(self, text):
        """
        Preprocess text by removing special characters, lowercasing, and lemmatizing.

        Parameters
        ----------
        text : str, list, or dict
            Text to be preprocessed

        Returns
        -------
        str, list, or dict
            Preprocessed text, or empty equivalent if input is None or invalid
        """
        def preprocess_string(text_str):
            if not isinstance(text_str, str):
                return ""
            text_str = re.sub(r"[^a-zA-Z0-9 _-]", "", text_str)
            text_str = text_str.lower().strip()
            words = word_tokenize(text_str)
            filtered_words = [self.lemmatizer.lemmatize(word) for word in words if word.lower() not in self.stop_words]
            return " ".join(filtered_words)

        if isinstance(text, str):
            return preprocess_string(text)
        elif isinstance(text, list):
            return [self.preprocess_text(item) for item in text]
        elif isinstance(text, dict):
            return {self.preprocess_text(key) if isinstance(key, str) else key: self.preprocess_text(value) if isinstance(value, str) else value
                    for key, value in text.items()}
        return ""

    def get_last_experience(self, experience_dict):
        """
        Get the most recent experience from user data.

        Parameters
        ----------
        experience_dict : list
            List of experience dictionaries

        Returns
        -------
        dict
            The most recent experience, or None if no valid experiences exist
        """
        if not experience_dict or not isinstance(experience_dict, list):
            return None
        start_date = ""
        last_exp = None
        for job in experience_dict:
            if isinstance(job, dict) and 'start_date' in job and isinstance(job['start_date'], str):
                if job['start_date'] > start_date:
                    start_date = job['start_date']
                    last_exp = job
        return last_exp

    def preprocess_object(self, users_df, job_df):
        """
        Preprocess object columns in DataFrames.

        Parameters
        ----------
        users_df : pandas.DataFrame
            User data DataFrame
        job_df : pandas.DataFrame
            Job data DataFrame

        Returns
        -------
        tuple
            (preprocessed_users_df, preprocessed_job_df)
        """
        if not job_df.empty:
            for col in job_df.columns:
                if job_df[col].dtype == 'object' and col != 'skills':
                    job_df[col] = job_df[col].apply(self.preprocess_text)

        if not users_df.empty:
            for col in users_df.columns:
                if users_df[col].dtype == 'object' and col != 'skills':
                    users_df[col] = users_df[col].apply(self.preprocess_text)

            users_df['last_experience'] = users_df['experiences'].apply(self.get_last_experience)
            users_df['location_type'] = users_df['last_experience'].apply(
                lambda x: x['location_type'] if x and isinstance(x, dict) and 'location_type' in x else ''
            )
            users_df['employment_type'] = users_df['last_experience'].apply(
                lambda x: x['employment_type'] if x and isinstance(x, dict) and 'employment_type' in x else ''
            )

        return users_df, job_df

    def preprocess_user_skills(self, skills):
        """
        Extract skill values from user skills data.

        Parameters
        ----------
        skills : list
            List of skill dictionaries

        Returns
        -------
        list
            List of skill values, or empty list if input is None or invalid
        """
        if not skills:
            return []
        return [value for d in skills for value in d.values() if isinstance(value, str)]

    def clean_and_tokenize(self, text):
        """
        Clean and tokenize text.

        Parameters
        ----------
        text : str
            Text to be cleaned and tokenized

        Returns
        -------
        list
            List of cleaned tokens, or empty list if input is None or invalid
        """
        if not isinstance(text, str):
            return []
        text = text.lower()
        text = re.sub(r'[^\w\s]', '', text)
        words = text.split()
        words = [word for word in words if word not in self.stop_words]
        return words

    def get_top_10_words(self, description):
        """
        Get the top 10 most common words in a description.

        Parameters
        ----------
        description : str
            Text description

        Returns
        -------
        list
            List of top 10 words, or empty list if input is None or invalid
        """
        if not isinstance(description, str):
            return []
        words = self.clean_and_tokenize(description)
        word_counts = Counter(words)
        top_10 = word_counts.most_common(10)
        return [word for word, count in top_10]

    def precompute_skill_embeddings(self, users_df, job_df):
        """
        Precompute embeddings for all skills in the dataset.

        Parameters
        ----------
        users_df : pandas.DataFrame
            User data DataFrame
        job_df : pandas.DataFrame
            Job data DataFrame

        Returns
        -------
        dict
            Dictionary mapping skills to embeddings
        """
        model = SentenceTransformer('all-MiniLM-L6-v2')
        all_skills = set()
        if not job_df.empty:
            for skills in job_df['skills']:
                if skills:
                    all_skills.update(skills)
        if not users_df.empty:
            for skills in users_df['skills']:
                if skills:
                    all_skills.update(skills)

        all_skills = list(all_skills)
        skill_embeddings = model.encode(all_skills, batch_size=128, show_progress_bar=True) if all_skills else {}
        skill_to_embedding = dict(zip(all_skills, skill_embeddings)) if all_skills else {}
        with open('skill_embeddings.pkl', 'wb') as f:
            pickle.dump(skill_to_embedding, f)
        return skill_to_embedding

    def precompute_title_embeddings(self, users_df, job_df):
        """
        Precompute embeddings for job titles and user subtitles.

        Parameters
        ----------
        users_df : pandas.DataFrame
            User data DataFrame
        job_df : pandas.DataFrame
            Job data DataFrame

        Returns
        -------
        dict
            Dictionary mapping titles/subtitles to embeddings
        """
        model = SentenceTransformer('all-MiniLM-L6-v2')
        all_titles = set()
        if not job_df.empty:
            for title in job_df['title']:
                if isinstance(title, str) and title.strip():
                    all_titles.add(title)
        if not users_df.empty:
            for subtitle in users_df['subtitle']:
                if isinstance(subtitle, str) and subtitle.strip():
                    all_titles.add(subtitle)

        all_titles = list(all_titles)
        title_embeddings = model.encode(all_titles, batch_size=128, show_progress_bar=True) if all_titles else {}
        title_to_embedding = dict(zip(all_titles, title_embeddings)) if all_titles else {}
        with open('title_embeddings.pkl', 'wb') as f:
            pickle.dump(title_to_embedding, f)
        return title_to_embedding

    def preprocess(self, input_json='test.json'):
        """
        Main preprocessing function to process input JSON data.

        Parameters
        ----------
        input_json : str or dict, optional
            Path to input JSON file or dictionary, by default 'test.json'

        Returns
        -------
        tuple
            (users_df, job_df, skill_embeddings, title_embeddings) Preprocessed DataFrames and embeddings
        """
        if isinstance(input_json, str):
            try:
                with open(input_json, 'r') as f:
                    input_data = json.load(f)
            except FileNotFoundError:
                print(f"Error: {input_json} not found")
                return pd.DataFrame(), pd.DataFrame(), {}, {}
        else:
            input_data = input_json

        if not isinstance(input_data, dict):
            print("Error: Input data must be a dictionary")
            return pd.DataFrame(), pd.DataFrame(), {}, {}

        # Handle 'job' or 'jobs' keys
        job_data = input_data.get('job', input_data.get('jobs', []))
        if isinstance(job_data, dict):
            job_data = [job_data]

        # Handle 'users' or 'user' keys
        user_data = input_data.get('users', input_data.get('user', []))
        if isinstance(user_data, dict):
            user_data = [user_data]

        # Filter users with valid about_me
        valid_users = []
        for user in user_data:
            if not isinstance(user, dict):
                continue
            about_me = user.get('about_me', "")
            if isinstance(about_me, str) and about_me.strip():
                valid_users.append({
                    'id': user.get('id', 0),
                    'about_me': about_me,
                    'subtitle': user.get('subtitle', ""),
                    'skills': user.get('skills', []),
                    'experiences': user.get('experiences', []),
                    'educations': user.get('educations', [])
                })

        users_df = pd.DataFrame(valid_users)
        job_df = pd.DataFrame(job_data)

        if not users_df.empty:
            users_df = users_df.drop_duplicates(subset=['about_me'], keep='first').reset_index(drop=True)
            users_df = users_df.iloc[:300].copy()
            users_df['skills'] = users_df['skills'].apply(self.get_user_skills)

        if not job_df.empty:
            job_df = job_df.drop_duplicates(subset=['description'], keep='first').reset_index(drop=True)
            job_df['skills'] = job_df['skills'].apply(self.parse_skills)

        users_df, job_df = self.preprocess_object(users_df, job_df)

        if not job_df.empty:
            job_df['top_10_words'] = job_df['description'].apply(self.get_top_10_words)
        if not users_df.empty:
            users_df['top_10_words'] = users_df['about_me'].apply(self.get_top_10_words)

        skill_embeddings = self.precompute_skill_embeddings(users_df, job_df)
        title_embeddings = self.precompute_title_embeddings(users_df, job_df)

        if not job_df.empty:
            job_df.to_pickle('job_df.pkl')
        if not users_df.empty:
            users_df.to_pickle('users_df.pkl')

        return users_df, job_df, skill_embeddings, title_embeddings


def preprocess(input_data):
    """
    Preprocess the input JSON data.

    Parameters
    ----------
    input_data : str or dict
        Path to input JSON file or dictionary

    Returns
    -------
    tuple
        (users_df, job_df, skill_embeddings, title_embeddings) Preprocessed DataFrames and embeddings
    """
    preprocessor = DataPreprocessor()
    return preprocessor.preprocess(input_data)

In [70]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
Job-to-user recommendation module.

This module provides functionality to recommend users for a job based on various
similarity metrics including title similarity, description similarity, skill matching,
employment type, and location type.
"""

import numpy as np
import pandas as pd
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


class UserRecommender:
    """
    A class for recommending users for a job.

    Provides methods to calculate various similarity metrics between a job and user profiles,
    and combines these metrics to generate recommendations.

    Attributes
    ----------
    title_weight : float
        Weight for title similarity in the final score
    cosine_weight : float
        Weight for description cosine similarity in the final score
    jaccard_weight : float
        Weight for skill similarity in the final score
    emp_type_weight : float
        Weight for employment type matching in the final score
    loc_type_weight : float
        Weight for location type matching in the final score
    top_n : int
        Number of recommendations to return
    title_embeddings : dict
        Precomputed title and subtitle embeddings
    """

    def __init__(self, title_embeddings, title_w=0.2, cosine_w=0.2, jaccard_w=0.2,
                 emp_type_w=0.2, loc_type_w=0.2, top_n=20):
        """
        Initialize the UserRecommender with scoring weights and title embeddings.

        Parameters
        ----------
        title_embeddings : dict
            Dictionary mapping titles/subtitles to embeddings
        title_w : float, optional
            Weight for title similarity, by default 0.2
        cosine_w : float, optional
            Weight for description similarity, by default 0.2
        jaccard_w : float, optional
            Weight for skill similarity, by default 0.2
        emp_type_w : float, optional
            Weight for employment type matching, by default 0.2
        loc_type_w : float, optional
            Weight for location type matching, by default 0.2
        top_n : int, optional
            Number of recommendations to return, by default 20
        """
        self.title_weight = title_w
        self.cosine_weight = cosine_w
        self.jaccard_weight = jaccard_w
        self.emp_type_weight = emp_type_w
        self.loc_type_weight = loc_type_w
        self.top_n = top_n
        self.title_embeddings = title_embeddings

    def compute_similarity(self, user_emb, job_emb):
        """
        Compute cosine similarity between user and job skill embeddings.

        Parameters
        ----------
        user_emb : numpy.ndarray
            User's skill embedding, shape (n_features,) or (1, n_features)
        job_emb : list or pandas.Series
            List or Series of job embeddings, each shape (n_features,)

        Returns
        -------
        numpy.ndarray
            Array of similarity scores for each job
        """
        if user_emb.ndim == 1:
            user_emb = user_emb.reshape(1, -1)

        job_emb = np.vstack(job_emb)

        if user_emb.shape[0] == 0 or job_emb.shape[0] == 0:
            return np.array([0] * len(job_emb))

        sim_matrix = cosine_similarity(user_emb, job_emb)
        return sim_matrix[0]

    def compute_common_skills(self, job_skills, user_skills, threshold=0.6):
        """
        Count job skills that have at least one user skill with similarity > threshold.

        Parameters
        ----------
        job_skills : list
            List of job skills (strings)
        user_skills : list
            List of user skills (strings)
        threshold : float, optional
            Cosine similarity threshold for a match, by default 0.6

        Returns
        -------
        int
            Number of common skills
        """
        if not user_skills or not job_skills:
            return 0

        try:
            with open('skill_embeddings.pkl', 'rb') as f:
                skill_to_embedding = pickle.load(f)
        except FileNotFoundError:
            print("Error: skill_embeddings.pkl not found")
            return 0

        user_embeddings = np.array([skill_to_embedding.get(skill, np.zeros(384)) for skill in user_skills])
        job_embeddings = np.array([skill_to_embedding.get(skill, np.zeros(384)) for skill in job_skills])

        sim_matrix = cosine_similarity(user_embeddings, job_embeddings)

        common_skills = sum(any(sim_matrix[i, j] > threshold for i in range(len(user_skills)))
                            for j in range(len(job_skills)))
        return common_skills

    def compute_title_similarity(self, job_title, user_subtitles):
        """
        Compute cosine similarity between a job title and user subtitles using embeddings.

        Parameters
        ----------
        job_title : str
            Job title
        user_subtitles : pandas.Series
            Series of user subtitles

        Returns
        -------
        numpy.ndarray
            Array of normalized similarity scores for each user subtitle
        """
        if not isinstance(job_title, str) or not job_title.strip():
            return np.zeros(len(user_subtitles))

        job_emb = self.title_embeddings.get(job_title, np.zeros(384))
        user_emb = np.array([self.title_embeddings.get(subtitle, np.zeros(384)) for subtitle in user_subtitles])

        similarities = cosine_similarity(job_emb.reshape(1, -1), user_emb)[0]
        max_similarity = similarities.max()
        normalized_similarities = similarities / max_similarity if max_similarity > 0 else similarities
        return normalized_similarities

    def recommend(self, job_df, users_df):
        """
        Generate user recommendations for a job.

        Parameters
        ----------
        job_df : pandas.DataFrame
            Job data DataFrame (single job)
        users_df : pandas.DataFrame
            Users data DataFrame

        Returns
        -------
        list
            List of recommended user dictionaries with 'id' and 'similarity_score'
        """
        if job_df.empty or users_df.empty:
            return []

        users_df = users_df.copy()
        job_df = job_df.iloc[0]

        # Handle None values
        users_df = users_df.fillna("")
        job_df = job_df.fillna("")

        # Calculate title similarity
        users_df['Title Similarity'] = self.compute_title_similarity(job_df['title'], users_df['subtitle'])

        # Calculate basic equality metrics
        users_df['Employee Equal'] = users_df['employment_type'].apply(
            lambda x: 1 if x == job_df['employee_type'] else 0)
        users_df['Location Equal'] = users_df['location_type'].apply(
            lambda x: 1 if x == job_df['location_type'] else 0)

        # Calculate description similarity using TF-IDF
        vectorizer = TfidfVectorizer(stop_words='english')
        job_vector = vectorizer.fit_transform([job_df['description']])
        user_vectors = vectorizer.transform(users_df['about_me'])

        raw_similarities = cosine_similarity(user_vectors, job_vector).flatten()
        max_score = raw_similarities.max()
        normalized_similarities = raw_similarities / max_score if max_score > 0 else raw_similarities
        users_df['about_me_similarity'] = normalized_similarities

        job_skills = set(job_df["skills"])
        users_df['common_skills'] = users_df['skills'].apply(
            lambda user_skills: self.compute_common_skills(job_skills, user_skills, 0.5)
        )

        # Normalize skill similarity
        max_common = users_df['common_skills'].max()
        users_df['skill_similarity'] = users_df['common_skills'] / max_common if max_common > 0 else 0.0

        # Calculate final score
        final_scores = (
            self.title_weight * users_df['Title Similarity'] +
            self.cosine_weight * users_df['about_me_similarity'] +
            self.emp_type_weight * users_df['Employee Equal'] +
            self.loc_type_weight * users_df['Location Equal'] +
            self.jaccard_weight * users_df['skill_similarity']
        )

        top_indices = final_scores.argsort()[::-1][:self.top_n]
        recommendations = [
            {
                "id": int(users_df.iloc[i]['id']),
                "similarity_score": float(final_scores.iloc[i])
            }
            for i in top_indices
        ]

        return recommendations


def recommend_for_user(job_df, users_df, title_embeddings, title_w=0.2, cosine_w=0.2, jaccard_w=0.2,
                       emp_type_w=0.2, loc_type_w=0.2, top_n=20):
    """
    Generate user recommendations for a job.

    Parameters
    ----------
    job_df : pandas.DataFrame
        Job data DataFrame (single job)
    users_df : pandas.DataFrame
        Users data DataFrame
    title_embeddings : dict
        Dictionary mapping titles/subtitles to embeddings
    title_w : float, optional
        Weight for title similarity, by default 0.2
    cosine_w : float, optional
        Weight for description similarity, by default 0.2
    jaccard_w : float, optional
        Weight for skill similarity, by default 0.2
    emp_type_w : float, optional
        Weight for employment type matching, by default 0.2
    loc_type_w : float, optional
        Weight for location type matching, by default 0.2
    top_n : int, optional
        Number of recommendations to return, by default 20

    Returns
    -------
    list
        List of recommended user dictionaries with 'id' and 'similarity_score'
    """
    if job_df.empty or users_df.empty:
        return []
    recommender = UserRecommender(
        title_embeddings=title_embeddings,
        title_w=title_w,
        cosine_w=cosine_w,
        jaccard_w=jaccard_w,
        emp_type_w=emp_type_w,
        loc_type_w=loc_type_w,
        top_n=top_n
    )
    return recommender.recommend(job_df, users_df)

In [71]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
Main script for the job-to-user recommendation system.

This script orchestrates the preprocessing of user and job data and generates
user recommendations for a specified job.
"""

def main(input_file='test.json'):
    """
    Main function to run the job-to-user recommendation system.

    Parameters
    ----------
    input_file : str, optional
        Path to input JSON file, by default 'test.json'

    Returns
    -------
    list
        List of recommended user dictionaries
    """
    # Preprocess data
    users_df, job_df, skill_embeddings, title_embeddings = preprocess(input_file)

    if job_df.empty or users_df.empty:
        print("No valid jobs or users found for recommendation")
        return []

    # Generate recommendations for the first job
    recommendations = recommend_for_user(
        job_df=job_df.iloc[[0]],  # Select first job
        users_df=users_df,
        title_embeddings=title_embeddings,
        title_w=0.2,
        cosine_w=0.2,
        jaccard_w=0.2,
        emp_type_w=0.2,
        loc_type_w=0.2,
        top_n=20
    )

    print("Recommended Users:", recommendations)
    return recommendations


if __name__ == "__main__":
    main()

Loaded stopwords from local file


Batches:   0%|          | 0/5 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Recommended Users: [{'id': 1238, 'similarity_score': 0.8000000029802323}, {'id': 1334, 'similarity_score': 0.7554570263624192}, {'id': 1213, 'similarity_score': 0.47545702636241916}, {'id': 1080, 'similarity_score': 0.4399408094834112}, {'id': 1124, 'similarity_score': 0.41096644210326905}, {'id': 1053, 'similarity_score': 0.39392401384818787}, {'id': 1281, 'similarity_score': 0.3830340440820479}, {'id': 1305, 'similarity_score': 0.3814196644972585}, {'id': 1154, 'similarity_score': 0.36359917163848876}, {'id': 1227, 'similarity_score': 0.35989759653806686}, {'id': 1237, 'similarity_score': 0.34757183241355644}, {'id': 1269, 'similarity_score': 0.34720982402563094}, {'id': 1187, 'similarity_score': 0.3454073944687843}, {'id': 1094, 'similarity_score': 0.3444330076956359}, {'id': 1043, 'similarity_score': 0.33730033576488494}, {'id': 1307, 'similarity_score': 0.33513206779956817}, {'id': 1200, 'similarity_score': 0.33116909668680106}, {'id': 1081, 'similarity_score': 0.3268416056036949}